## Simultaion des données et création d'une base de données SQLite

# Sommaire
* [1. Simulation des données](#10)
    * [1.1. Création des données clients](#11)
    * [1.2. Création des données achats agrégés](#12)
    * [1.3. Création des données des tickets de caisse](#13)

* [2. Création de la base de données SQlite](#20)
    * [2.1. Créer une base de données SQLite et insérer les données](#21)
    * [2.2. Vérification des données dans la base de données SQLite](#22)

# 1. Simulation des données<a class = 'anchor' id = '10'></a>


## 1.1. Création de la base de données clients<a class = 'anchor' id = '11'></a>

<div style = "text-align: justify">Étant donné que le projet est fictif, on va devoir créer <em>ex nihilo</em> une base de données clients. Tâchons toutefois de la rendre la plus réaliste possible. L'objectif ici va donc être de créer une base de données contenant trois millions de lignes (la taille estimée du fichier clients de l'entreprise GoldenLine) grâce au <em>package</em> <tt>pandas</tt>. Celle-ci devra comporter les colonnes suivantes :<ol>
    <li><tt>NOM</tt> : le nom du client, extrait aléatoirement de la base des patronymes (nom de famille) utilisés en France, accessible depuis le site du gouvernement français au format <tt>CSV</tt> <a href="https://www.data.gouv.fr/fr/datasets/r/9ae80de2-a41e-4282-b9f8-61e6850ef449">ici</a>. Le nom de famille sera sélectionné parmi les noms utilisés au moins cent fois dans la base de données (patronyme "réaliste").</li>
   <li><tt>PRENOM</tt> : le prénom du client, extrait de la base des prénoms utilisés en France depuis le site du gouvernement français accessible au format <tt>CSV</tt> <a href="https://www.data.gouv.fr/fr/datasets/r/4b13bbf2-4185-4143-92d3-8ed5d990b0fa">ici</a>. Le prénom sera sélectionné parmi les noms utilisés au moins cent fois dans la base de données (prénom courant).</li>
   <li><tt>MAIL</tt> : l'adresse mail du client, que l'on simulera sous la forme <tt>prenom.nom@gmail.com</tt> ; on aurait pu utiliser d'autres fournisseurs d'accès mais cela ne rajouterait que de la complexité inutile.</li>
   <li><tt>ENFANTS</tt> : le nom d'enfants du client, qu'on supposera suivre une loi binomiale de paramètres 12 (on supposera en effet qu'il est difficile d'en avoir plus) et 0.1 (de sorte à ce que le nombre moyen d'enfants par personne vérifie 12 $\times$ 0.1 = 1.2, valeur réaliste).</li>
   <li><tt>CSP</tt> : la catégorie socioprofessionnelle (CSP) du client, selon la répartition suivante : 1 % agriculteurs exploitants, 5 % artisans, commerçants et chefs d'entreprise, 13 % de cadres, 13 % de professions intermédiaires, 8 % d'employés, 18 % d'ouvriers, 30 % de retraités, 12 % d'étudiants.</li>
</ol></div>

<div style = "text-align: justify">Quelles sont les limites d'une telle simulation ? Tout d'abord, on ne simule pas les âges de nos clients par rapport à l'entreprise : les entreprises de la tech ont naturellement des clients plus jeunes que celles de la grande distribution. Dès lors, la répartition des CSP au sein de notre fichier client, que l'on a supposé suivre celle de la population française générale, n'a pas de raison <em>a priori</em> de correspondre à celle de GoldenLine. Ceci étant dit, commençons par récupérer les bases de données des patronymes et des prénoms depuis le site du gouvernement.</div>

In [54]:
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import os
import random
from datetime import datetime, timedelta
import warnings                   # Empêche l'affichage de messages d'avertissement
warnings.filterwarnings('ignore') # qui brouillent la lecture du notebook Jupyter

In [55]:
%%time

to_scrap = ['https://www.data.gouv.fr/fr/datasets/r/9ae80de2-a41e-4282-b9f8-61e6850ef449', 'https://www.data.gouv.fr/fr/datasets/r/4b13bbf2-4185-4143-92d3-8ed5d990b0fa']
database = ['NOMS', 'PRENOMS']
nb_min   = 100

for i in range(len(to_scrap)):
    db = pd.read_csv(to_scrap[i], sep = ',')
    db.columns = ['NOM', 'NOMBRE']
    db = db[db.NOMBRE >= nb_min]
    db.to_csv('%s.csv' %database[i])

CPU times: total: 219 ms
Wall time: 2.5 s


<div style = "text-align: justify">Maintenant qu'on a récupéré les bases de données de noms et prénoms relativement courants, on va créer notre base de données de trois millions de lignes à partir des règles définies <em>supra</em>. Comme on va réaliser des simulations probabilistes, on aura besoin du <em>package</em> <tt>numpy</tt>. En particulier, la stratégie de création des noms est la suivante : on génère un vecteur aléatoire de trois millions d'entiers <tt>[n_0,...,n_k]</tt> compris entre 0 et la taille de la base des noms (moins 1, car <tt>Python</tt> commence à compter à partir de 0), nommée <tt>noms</tt>, puis on génère une liste aléatoire des prénoms par <tt>[noms[n_0],...noms[n_k]]</tt>. On procède de façon analogue pour générer aléatoirement des prénoms. Quid de la répartition des CSP qui doivent être aléatoirement générés ? Pour ce faire, on va générer trois millions de réels compris entre 0 et 1 comme une suite de variables aléatoires $U_0,\ldots,U_{2~999~999}$ identiquement distribués selon une loi uniforme sur l'intervalle [0,1]. On sait alors que, pour tous réels $x$ et $y$ compris entre 0 et 1 vérifiant $x\leq y$, on a $\mathbb{P}([x \leq U\leq y])=y-x$ (où $U$ est une variable aléatoire suivant une loi uniforme sur [0,1]). Par conséquent, la stratégie de simulation des CSP sera la suivante, pour tout individu $i$ compris entre 0 et 2 999 999 : <ol>
    <li>Si $U_i$ est inférieur à 0.01, ce qui survient dans 0.01 $-$ 0 = 1 % des cas, on supposera que l'individu $i$ est un agriculeur ;</li>
    <li>Si $U_i$ est compris entre 0.01 et 0.06, ce qui survient dans 0.06 $-$ 0.01 = 5 % des cas, on supposera que l'individu $i$ est un artisan ou apparenté ;</li>
    <li>Si $U_i$ est compris entre 0.06 et 0.19, ce qui survient dans 0.19 $-$ 0.06 = 13 % des cas, on supposera que l'individu $i$ est un cadre ;</li>
    <li>Si $U_i$ est compris entre 0.19 et 0.32, ce qui survient dans 0.32 $-$ 0.19 = 13 % des cas, on supposera que l'individu $i$ est une profession intermédiaire ;</li>
    <li>Si $U_i$ est compris entre 0.32 et 0.40, ce qui survient dans 0.40 $-$ 0.32 = 8 % des cas, on supposera que l'individu $i$ est un employé ;</li> 
    <li>Si $U_i$ est compris entre 0.40 et 0.58, ce qui survient dans 0.58 $-$ 0.40 = 18 % des cas, on supposera que l'individu $i$ est un ouvrier ;</li> 
    <li>Si $U_i$ est compris entre 0.58 et 0.88, ce qui survient dans 0.88 $-$ 0.58 = 30 % des cas, on supposera que l'individu $i$ est un retraité ;</li> 
    <li>Si $U_i$ est supérieur à 0.88, ce qui survient dans 1 $-$ 0.88 = 12 % des cas, on supposera que l'individu $i$ est un étudiant.</li>     
</ol></div>

In [56]:
%%time

col1     = ['NOM', 'PRENOM', 'MAIL', 'ENFANTS', 'CSP']
client   = pd.DataFrame(columns = col1)
taille   = 20000

noms     = pd.read_csv('NOMS.csv')
noms     = noms.NOM
nom_pos  = np.random.randint(len(noms) - 1, size = taille)
nom_sim  = [noms[i] for i in nom_pos]

prenoms  = pd.read_csv('PRENOMS.csv')
prenoms  = prenoms.NOM
pre_pos  = np.random.randint(len(prenoms) - 1, size = taille)
pre_sim  = [prenoms[i].capitalize() for i in pre_pos]

csp      = np.random.rand(taille)
repart   = [csp < 0.01, (csp >= 0.01) & (csp < 0.06), (csp >= 0.06) & (csp < 0.19), (csp >= 0.19) & (csp < 0.32), (csp >= 0.32) & (csp < 0.40), (csp >= 0.40) & (csp < 0.58), (csp >= 0.58) & (csp < 0.88), csp >= 0.88]
csp_list = ['Agriculteur', 'Artisan et apparentés', 'Cadre', 'Profession intermédiaire', 'Employé', 'Ouvrier', 'Retraité', 'Étudiant']
csp_sim  = np.select(repart, csp_list, 'Non renseigné')

client['NOM']     = nom_sim
client['PRENOM']  = pre_sim
client['MAIL']    = client['PRENOM'].apply(str.lower).str.replace(' ', '-') + '.' + client['NOM'].apply(str.lower).str.replace(' ', '') + '@gmail.com'
client['ENFANTS'] = np.random.binomial(12, 0.1, taille)
client['CSP']     = csp_sim

client.head(10)

CPU times: total: 109 ms
Wall time: 364 ms


,NOM,PRENOM,MAIL,ENFANTS,CSP
0,RICART,Charl,charl.ricart@gmail.com,1,Artisan et apparentés
1,GUENARD,Orianne,orianne.guenard@gmail.com,2,Ouvrier
2,PENEL,Cosimo,cosimo.penel@gmail.com,0,Cadre
3,FARDEAU,Marlene,marlene.fardeau@gmail.com,0,Profession intermédiaire
4,DIOUF,Annik,annik.diouf@gmail.com,2,Retraité
5,ROUAN,George,george.rouan@gmail.com,2,Ouvrier
6,HELD,Timothee,timothee.held@gmail.com,3,Étudiant
7,HELLEGOUARCH,Sarah,sarah.hellegouarch@gmail.com,2,Retraité
8,FAUX,Malcolm,malcolm.faux@gmail.com,1,Cadre
9,SURREL,Svetlana,svetlana.surrel@gmail.com,0,Étudiant


<div style = "text-align: justify">Reste à anonymiser la base de données clients. On recourera pour cela au <em>package</em> <tt>hashlib</tt> qui nous permettra de créer un identifiant client unique à partir de l'adresse mail, qu'on sauvegardera dans une colonne <tt>ID</tt> de la base de données clients. Cette stratégie d'anonymisation est bien performante. En effet, le <em>package</em> <tt>hashlib</tt> recourt à l'algorithme SHA256 qui est asymétrique : si on connaît une valeur $x$, on peut en déduire la valeur $y$ hashée de $x$ ; en revanche, si l'on connaît $y$, on ne peut remonter à la valeur initiale de $x$ (cela repose sur le fait que la "fonction" de hashage est en réalité une correspondance et n'est donc pas bijective). Attention, cette base de données nous permet de rapprocher chaque achat d'une personne "réelle", puisque la base clients comportera à la fois le nom, le prénom et l'identifiant : il ne faudra donc pas que cette base de données soit accessible à tous les services. On fournira alors une base de données anonymisée dans laquelle seront supprimées les deux premières colonnes.</div>

In [57]:
%%time

client['ID'] = client['MAIL'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
anonyme      = client[['ID', 'ENFANTS', 'CSP']]
#client.to_csv('Base de données clients complète.csv')
#anonyme.to_csv('Base de données clients anonymisée.csv')

anonyme.head(10)

CPU times: total: 31.2 ms
Wall time: 87.7 ms


,ID,ENFANTS,CSP
0,7ce3cb5fb3bdea11a528088c0ee1278178030122b4ee57...,1,Artisan et apparentés
1,c8e52d98c3ce008961f0fc097232295573542c425ed961...,2,Ouvrier
2,6204a7d0e3e874123dce0cc718d767019f037dbfa8b8f2...,0,Cadre
3,2dd4b3fdfad94d3fbcf1d8a702da16991aa991d87265c5...,0,Profession intermédiaire
4,7719884d3944fb9b1ca70571cb31c808b2f9471b5cf9fb...,2,Retraité
5,89e75a0dcaa4c853f009dc9b1e53eb27461e285618809f...,2,Ouvrier
6,67e834b91bc92f45bd772ce4290a6e09294b3ec15d2b01...,3,Étudiant
7,d13435d26614ce22baf4fc434d30ee1fe6317adf2d3773...,2,Retraité
8,091293096737a41b67b066b05fdd4008331706897ba76f...,1,Cadre
9,150aa757253266d7b2e9f0fa9ff3c56e6a724d0ea6c9e0...,0,Étudiant


## 1.2. Création des données achats agrégées<a class = 'anchor' id = '12'></a>

<div style = "text-align: justify">L'objectif est à présent de simuler des factures. La première étape est de simuler le nombre d'achats réalisés pour chaque client de la base de données par année (par exemple). Imaginons que ce nombre correspondent à une loi binomiale de paramètres 5 et 0.1 (de sorte à ce qu'un client achète une fois tous les deux ans en moyenne). La première des étapes est donc de générer un vecteur correspondant au nombre de fois où chaque client apparaît dans la base de données.</div>

In [58]:
%%time
ID = []

for client_id in anonyme['ID']:
    ID.extend([client_id for i in range(np.random.binomial(5, 0.1, 1)[0])])

CPU times: total: 93.8 ms
Wall time: 128 ms


<div style = "text-align: justify">C'est ce vecteur d'apparition client qui va définir le nombre de lignes de notre base de données d'achat. On va tout d'abord générer un identifiant de collecte (c'est-à-dire une sorte de numéro de facture) de façon aléatoire. On considèrera que cet identifiant de collecte est un unique entier aléatoire entre 10 000 000 et 99 999 999 ; pour cela, on utilisera le <em>package</em> <tt>random</tt>. On pourra alors, pour chacune des lignes, générer un prix fictif du ticket de caisse, qu'on supposera suivre une loi normale d'espérance 70 (panier moyen supposé des achats) et d'écart-type 10 (et dont on imposera qu'il prenne une valeur positive). Naturellement, cette valeur-prix devra être arrondie au cent près. On supposera que les identifiants de collecte permettent de classer chronologiquement les opérations : un identifiant de collecte plus élevé correspondra à un achat plus récent. Lorsqu'on générera la base de données des achats, on choisira de les classer par ordre chronologique croissant. Pour plus de vraisemblance, à chaque numéro de facture, on associera une date fictive et aléatoire comprise entre début janvier 2020 et la date du jour. Attention, bien que la date ainsi créée soit aléatoire (et obtenue grâce au <em>package</em> <tt>datetime</tt> qui permet la gestion du format date), plus l'identifiant de collecte est bas, plus la date doit être antérieure à la date du jour : deux factures ayant des numéros $A$ et $B$ (avec $A>B$) dont les dates respectives sont $d_A$ et $d_B$ devra nécessairement respecter la contrainte $d_A\geq d_B$.</div>

In [59]:
%%time

start_date = datetime(2020, 1, 1)
end_date = datetime.now().date()
date_range = pd.date_range(start_date, end_date)
random_dates = random.choices(date_range, k=len(ID))
collecte = random.sample(range(10000000, 99999999), len(ID))
prix     = abs(np.random.normal(70, 10, len(ID)).round(2))
col2     = ['COLLECTE', 'PRIX', 'ID','DATE']
achats   = pd.DataFrame(columns = col2)
achats['COLLECTE'] = collecte
achats['PRIX']     = prix
achats['ID']       = ID
achats['DATE']     = random_dates
achats.sort_values(by = ['COLLECTE'], ignore_index = True, inplace = True)
#achats.to_csv('Base de données achats.csv')


achats.head(10)

CPU times: total: 31.2 ms
Wall time: 199 ms


,COLLECTE,PRIX,ID,DATE
0,10001227,82.28,b2bc1fcfa94dd969191b4703e839b2f88c158c5a1d7062...,2021-02-21
1,10002649,61.86,62cfff947f04af5e2a8faecc1b74181555f00d16782177...,2022-03-12
2,10004285,58.91,e6c2f4f93db502cea3380ca2e451977cd1f06b640fb378...,2022-01-13
3,10018382,67.50,a68493328756e4c1a678c65900a5f279a108848e04a596...,2020-03-22
4,10040695,90.06,5ce90f37825f002b253934d5b4636e1639a5610e47b1dc...,2022-03-02
5,10054488,73.76,87fed246986fe29f94c0d39714f81cdd093c6947df91a6...,2020-07-20
6,10063913,56.03,604dd5e2447bd541a4d5ba5da21528ac7e8e013ac613d5...,2024-04-17
7,10072319,60.76,a3bb50f6ea7b9d556c9b847f4ec65afa5f146776335b81...,2022-01-26
8,10072980,83.74,4e9a8f056dc120f97ff68bf0550930150d51355e6387ad...,2022-03-11
9,10075612,78.47,3ddde56a78bb397f948e15e61cb4956fcbbb75ad996ccf...,2020-01-27


## 1.3. Création des données des tickets de caisse<a class = 'anchor' id = '13'></a>

<div style = "text-align: justify">On doit pour finir générer le <em>breakdown</em> de chaque ticket de caisse en fonction des différentes catégories de produits vendus dans le magasin, spécialisé rappelons-le dans le vente de vêtements. On va distinguer entre quatre catégories : alimentaire, vêtements, maison et hygiène, qu'on suppose être les secteurs de prédilection de GoldenLine. La stratégie de génération aléatoire des répartitions des achats est la suivante : on va générer, pour chaque ticket de caisse, quatre nombres aléatoires $x_1$, $x_2$, $x_3$ et $x_4$ compris entre 0 et 1 d'après une loi uniforme sur [0,1]. Dès lors, $x_1/(x_1+\cdots+x_4)$ (resp. $x_2/(x_1+\cdots+x_4)$, $x_3/(x_1+\cdots+x_4)$ et $x_4/(x_1+\cdots+x_4)$) représentera la part de l'alimentaire (resp. des vêtements, des produits liés à la maison et des produits d'hygiène) dans le ticket de caisse. En effet, la somme de ces quatre noms valant 1, on peut bien les assimiler à des pondérations.</div>

In [60]:
%%time
x1 = np.random.rand(len(ID))
x2 = np.random.rand(len(ID))
x3 = np.random.rand(len(ID))
x4 = np.random.rand(len(ID))

details = achats[['COLLECTE', 'PRIX']]
details['ALIMENTAIRE'] = (x1 * details['PRIX'] / (x1 + x2 + x3 + x4)).round(2)
details['VETEMENTS']   = (x2 * details['PRIX'] / (x1 + x2 + x3 + x4)).round(2)
details['MAISON']      = (x3 * details['PRIX'] / (x1 + x2 + x3 + x4)).round(2)
details['HYGIENE']     = details['PRIX'] - details['ALIMENTAIRE'] - details['VETEMENTS'] - details['MAISON']
details = details.drop(['PRIX'], axis = 1)
#details.to_csv('Base de données détails des factures.csv')

details.head(10)

CPU times: total: 0 ns
Wall time: 15.3 ms


,COLLECTE,ALIMENTAIRE,VETEMENTS,MAISON,HYGIENE
0,10001227,33.65,29.01,12.57,7.05
1,10002649,14.06,3.88,19.84,24.08
2,10004285,18.96,12.57,16.29,11.09
3,10018382,10.64,13.13,14.26,29.47
4,10040695,59.88,8.05,17.53,4.60
5,10054488,27.47,4.56,18.62,23.11
6,10063913,18.39,13.32,1.61,22.71
7,10072319,0.57,14.69,23.38,22.12
8,10072980,29.16,25.55,15.71,13.32
9,10075612,34.75,18.53,23.69,1.50


<div style = "text-align: justify">Pour la base de données des détails des factures, de rapprocher l'identifiant client à chaque identifiant de collecte grâce à la base de données des achats puis, étant donné qu'on dispose de l'identifiant client, de rapprocher le nombre d'enfants et la CSP à l'aide de la base de données clients anonymisée.</div>

In [61]:
%%time
complete = details.merge(achats,   left_on = 'COLLECTE', right_on = 'COLLECTE')
complete = complete.merge(anonyme, left_on = 'ID',       right_on = 'ID')
complete = complete[['DATE', 'COLLECTE', 'ID', 'CSP', 'ENFANTS', 'PRIX', 'ALIMENTAIRE', 'VETEMENTS', 'MAISON', 'HYGIENE']]
complete.sort_values(by = ['COLLECTE'], ignore_index = True, inplace = True)
#complete.to_csv('Base de données complète anonymisée.csv')
complete.head(10)

CPU times: total: 0 ns
Wall time: 50.9 ms


,DATE,COLLECTE,ID,CSP,ENFANTS,PRIX,ALIMENTAIRE,VETEMENTS,MAISON,HYGIENE
0,2021-02-21,10001227,b2bc1fcfa94dd969191b4703e839b2f88c158c5a1d7062...,Cadre,1,82.28,33.65,29.01,12.57,7.05
1,2022-03-12,10002649,62cfff947f04af5e2a8faecc1b74181555f00d16782177...,Ouvrier,0,61.86,14.06,3.88,19.84,24.08
2,2022-01-13,10004285,e6c2f4f93db502cea3380ca2e451977cd1f06b640fb378...,Retraité,2,58.91,18.96,12.57,16.29,11.09
3,2020-03-22,10018382,a68493328756e4c1a678c65900a5f279a108848e04a596...,Ouvrier,2,67.50,10.64,13.13,14.26,29.47
4,2022-03-02,10040695,5ce90f37825f002b253934d5b4636e1639a5610e47b1dc...,Étudiant,1,90.06,59.88,8.05,17.53,4.60
5,2020-07-20,10054488,87fed246986fe29f94c0d39714f81cdd093c6947df91a6...,Employé,2,73.76,27.47,4.56,18.62,23.11
6,2024-04-17,10063913,604dd5e2447bd541a4d5ba5da21528ac7e8e013ac613d5...,Cadre,1,56.03,18.39,13.32,1.61,22.71
7,2022-01-26,10072319,a3bb50f6ea7b9d556c9b847f4ec65afa5f146776335b81...,Ouvrier,0,60.76,0.57,14.69,23.38,22.12
8,2022-03-11,10072980,4e9a8f056dc120f97ff68bf0550930150d51355e6387ad...,Retraité,0,83.74,29.16,25.55,15.71,13.32
9,2020-01-27,10075612,3ddde56a78bb397f948e15e61cb4956fcbbb75ad996ccf...,Étudiant,0,78.47,34.75,18.53,23.69,1.50


# 2. Création de la base de données SQlite<a class = 'anchor' id = '20'></a>

## 2.1. Créer une base de données SQLite et insérer les données<a class = 'anchor' id = '21'></a>

<div style = "text-align: justify">Nous n'avons plus qu'à créer une connection à la la base de données SQLite 'my_database.db' et insérer chaque Dataframe dans un table SQLite</div>

In [62]:
dfs = [client, achats, details, complete]
table_names = ['client', 'achats', 'details', 'complete']

# Afficher les premières lignes de chaque DataFrame pour vérifier le chargement des données
for name, df in zip(table_names, dfs):
    print(f"DataFrame '{name}' :")
    print(df.head(), "\n")

# Créer une connexion à la base de données SQLite
conn = sqlite3.connect('my_database.db')

# Insérer chaque DataFrame dans une table SQLite
for name, df in zip(table_names, dfs):
    df.to_sql(name, conn, if_exists='replace', index=False)
    print(f"Table '{name}' créée avec succès.")

conn.close()

DataFrame 'client' :


       NOM   PRENOM                       MAIL  ENFANTS  \
0   RICART    Charl     charl.ricart@gmail.com        1   
1  GUENARD  Orianne  orianne.guenard@gmail.com        2   
2    PENEL   Cosimo     cosimo.penel@gmail.com        0   
3  FARDEAU  Marlene  marlene.fardeau@gmail.com        0   
4    DIOUF    Annik      annik.diouf@gmail.com        2   

                        CSP                                                 ID  
0     Artisan et apparentés  7ce3cb5fb3bdea11a528088c0ee1278178030122b4ee57...  
1                   Ouvrier  c8e52d98c3ce008961f0fc097232295573542c425ed961...  
2                     Cadre  6204a7d0e3e874123dce0cc718d767019f037dbfa8b8f2...  
3  Profession intermédiaire  2dd4b3fdfad94d3fbcf1d8a702da16991aa991d87265c5...  
4                  Retraité  7719884d3944fb9b1ca70571cb31c808b2f9471b5cf9fb...   

DataFrame 'achats' :
   COLLECTE   PRIX                                                 ID  \
0  10001227  82.28  b2bc1fcfa94dd969191b4703e839b2f88c158c5a1d7

## 2.2. 2.2. Vérification des données dans la base de données SQLite<a class = 'anchor' id = '11'></a>

<div style = "text-align: justify">Enfin, pour vérifier que les données ont été correctement insérées, nous allons lire les tables depuis la base de données SQLite et afficher les premières lignes</div>

In [63]:
conn = sqlite3.connect('my_database.db')

for name in table_names:
    df_from_db = pd.read_sql(f'SELECT * FROM {name}', conn)
    print(f"DataFrame '{name}' from SQLite :")
    print(df_from_db.head(), "\n")

conn.close()

DataFrame 'client' from SQLite :
       NOM   PRENOM                       MAIL  ENFANTS  \
0   RICART    Charl     charl.ricart@gmail.com        1   
1  GUENARD  Orianne  orianne.guenard@gmail.com        2   
2    PENEL   Cosimo     cosimo.penel@gmail.com        0   
3  FARDEAU  Marlene  marlene.fardeau@gmail.com        0   
4    DIOUF    Annik      annik.diouf@gmail.com        2   

                        CSP                                                 ID  
0     Artisan et apparentés  7ce3cb5fb3bdea11a528088c0ee1278178030122b4ee57...  
1                   Ouvrier  c8e52d98c3ce008961f0fc097232295573542c425ed961...  
2                     Cadre  6204a7d0e3e874123dce0cc718d767019f037dbfa8b8f2...  
3  Profession intermédiaire  2dd4b3fdfad94d3fbcf1d8a702da16991aa991d87265c5...  
4                  Retraité  7719884d3944fb9b1ca70571cb31c808b2f9471b5cf9fb...   



DataFrame 'achats' from SQLite :
   COLLECTE   PRIX                                                 ID  \
0  10001227  82.28  b2bc1fcfa94dd969191b4703e839b2f88c158c5a1d7062...   
1  10002649  61.86  62cfff947f04af5e2a8faecc1b74181555f00d16782177...   
2  10004285  58.91  e6c2f4f93db502cea3380ca2e451977cd1f06b640fb378...   
3  10018382  67.50  a68493328756e4c1a678c65900a5f279a108848e04a596...   
4  10040695  90.06  5ce90f37825f002b253934d5b4636e1639a5610e47b1dc...   

                  DATE  
0  2021-02-21 00:00:00  
1  2022-03-12 00:00:00  
2  2022-01-13 00:00:00  
3  2020-03-22 00:00:00  
4  2022-03-02 00:00:00   

DataFrame 'details' from SQLite :
   COLLECTE  ALIMENTAIRE  VETEMENTS  MAISON  HYGIENE
0  10001227        33.65      29.01   12.57     7.05
1  10002649        14.06       3.88   19.84    24.08
2  10004285        18.96      12.57   16.29    11.09
3  10018382        10.64      13.13   14.26    29.47
4  10040695        59.88       8.05   17.53     4.60 

DataFrame 'complete' f